In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from  sklearn.neighbors import KNeighborsClassifier
import spacy
from sklearn.metrics import classification_report, jaccard_score, accuracy_score, confusion_matrix, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("train.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

(404290, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df = df.dropna(subset=['question1', 'question2'])

In [ ]:
# Combine 'question1' and 'question2' into a single string for each row
df['combined'] = df['question1'].astype(str) + ' ' + df['question2'].astype(str)

In [ ]:
#check the distribution of labels
df['is_duplicate'].value_counts()

,count
is_duplicate,
0,255024
1,149263


In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,combined
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,How can I increase the speed of my internet co...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,Why am I mentally very lonely? How can I solve...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"Which one dissolve in water quikly sugar, salt..."


In [ ]:
df= df.sample(n=20000, random_state=42)

In [ ]:
df['vector'] = df['combined'].apply(lambda text: nlp(text).vector)

In [ ]:
df['vector'].iloc[2].shape

(300,)

In [ ]:
df.to_csv("vectors.csv")

In [ ]:
# Prepare feature matrix X by concatenating vector1 and vector2
X = df['vector'].values

# Prepare target vector Y
Y = df['is_duplicate'].values

In [ ]:
X = np.stack(X)

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions (e.g., from 600 to 200 components)
pca = PCA(n_components=150)
X = pca.fit_transform(X)

# Check the new shape of X
print(X.shape)  # Output will be (n_samples, 200)

(20000, 150)


In [ ]:
# Split data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train)
scaled_test_embed = scaler.transform(X_test)

In [ ]:
clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      1.00      0.77      2522
           1       0.00      0.00      0.00      1478

    accuracy                           0.63      4000
   macro avg       0.32      0.50      0.39      4000
weighted avg       0.40      0.63      0.49      4000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#1. creating a KNN model object
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')

#2. fit with all_train_embeddings and y_train
clf.fit(X_train, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      2522
           1       0.64      0.49      0.56      1478

    accuracy                           0.71      4000
   macro avg       0.69      0.67      0.67      4000
weighted avg       0.70      0.71      0.70      4000



In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

rf_pred = rf_clf.predict(X_test)

print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.70      0.94      0.80      2522
           1       0.75      0.32      0.44      1478

    accuracy                           0.71      4000
   macro avg       0.72      0.63      0.62      4000
weighted avg       0.72      0.71      0.67      4000



In [ ]:
# Train SVM Classifier
svm_clf = SVC(random_state=42)
svm_clf.fit(X_train, y_train)

svm_pred = svm_clf.predict(X_test)

print(classification_report(y_test, svm_pred))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81      2522
           1       0.70      0.49      0.58      1478

    accuracy                           0.74      4000
   macro avg       0.73      0.69      0.69      4000
weighted avg       0.73      0.74      0.72      4000



In [ ]:
# Initialize Logistic Regression model
logistic_reg = LogisticRegression(max_iter=1000)

# Train the model
logistic_reg.fit(X_train, y_train)

# Make predictions on the test set
Y_pred = logistic_reg.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.71      0.87      0.78      2522
           1       0.64      0.40      0.49      1478

    accuracy                           0.69      4000
   macro avg       0.67      0.63      0.63      4000
weighted avg       0.68      0.69      0.67      4000



In [ ]:
# XGBoost Classifier
classifier = XGBClassifier(verbose=1)

# Fit the model
classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:51:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.73      0.84      0.78      2522
           1       0.64      0.47      0.54      1478

    accuracy                           0.71      4000
   macro avg       0.69      0.66      0.66      4000
weighted avg       0.70      0.71      0.69      4000



In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['combined'], Y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# Logistic Regression
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Predictions
predictions = classifier.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(confusion_matrix(y_test, predictions))

Accuracy: 0.726
F1 Score: 0.7065860616448305
Recall: 0.726
[[2265  257]
 [ 839  639]]


In [ ]:
from xgboost import XGBClassifier

# XGBoost Classifier
classifier = XGBClassifier(verbose=1)

# Fit the model
classifier.fit(X_train_tfidf, y_train)

# Predictions on the test set
predictions = classifier.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(confusion_matrix(y_test, predictions))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:51:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.72175
F1 Score: 0.710317514774821
Recall: 0.72175
[[2164  358]
 [ 755  723]]


In [3]:
# Tokenize the sentences
tokenized_sentences_q1 = df['question1'].apply(lambda x: str(x).split())
tokenized_sentences_q2 = df['question2'].apply(lambda x: str(x).split())

# Train Word2Vec model
word2vec_model_q1 = Word2Vec(sentences=tokenized_sentences_q1, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model_q2 = Word2Vec(sentences=tokenized_sentences_q2, vector_size=100, window=5, min_count=1, workers=4)

# Function to get the vector representation of a sentence
def get_sentence_vector(sentence, model):
    vector = np.zeros(model.vector_size)
    count = 0
    for word in sentence:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

# Create feature vectors for each question
X_q1 = np.array([get_sentence_vector(sentence, word2vec_model_q1) for sentence in tokenized_sentences_q1])
X_q2 = np.array([get_sentence_vector(sentence, word2vec_model_q2) for sentence in tokenized_sentences_q2])

In [4]:

# Concatenate the feature vectors
X_combined = np.concatenate((X_q1, X_q2), axis=1)
# Prepare target vector Y
Y = df['is_duplicate'].values
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)

# Random Forest without tuning
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

# Predictions
y_pred = rfc.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
# f1 = f1_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# print(f"F1 Score: {f1}")
# print(f"Recall: {recall}")

Accuracy: 0.795307823591976
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.92      0.85     50803
           1       0.81      0.59      0.68     30055

    accuracy                           0.80     80858
   macro avg       0.80      0.75      0.77     80858
weighted avg       0.80      0.80      0.79     80858

